# Achate a curva [in Portuguese]
> Número de casos ativos e estimativa de leitos de UTI total e para pacientes covid-19

- toc: false
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [covid-19, brasil, comparativo, altair, jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from altair_saver import save
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

In [2]:
#hide
url_icus = ('https://covid-insumos.saude.gov.br/paineis/insumos/lista_csv_painel.php?output=csv')
src_icus = pd.read_csv(url_icus, sep=';')

In [3]:
#hide
STATE_COLUMN = "Estado"
ICUS_COLUMN = 'UTIs'
AVAILABLE_PERCENTAGE_ICU = 0.35
src_icus = src_icus.rename(columns={'Leitos UTI adulto':ICUS_COLUMN, 'uf':STATE_COLUMN})

In [4]:
#hide
TOTAL_ICU = src_icus[ICUS_COLUMN].sum()
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU

In [5]:
#hide
DATE_COLUMN = "Data"
CONFIRMED_CASES = 'Casos confirmados'
VALUE = 'Valor'
ACTIVE_CASES = 'Ativos'
CASES_NEED_ICU = 0.05
ICU_PER_100k = 20
CASES_NEED_ICU_COLUMN = "{:.0f}".format(CASES_NEED_ICU*100)+"% dos ativos"
PATIENTS = "Pacientes"
AVAILABLE_ICUS_COLUMN = "{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% das UTIs"
UPPER_BOUND_ICU_NEED = 0.12
LOWER_BOUND_ICU_NEED = 0.01
UPPER_BOUND_ICU_NEED_STR = "{:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"% dos ativos"
LOWER_BOUND_ICU_NEED_STR = "{:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"% dos ativos"

In [6]:
#hide
url = ('https://data.brasil.io/dataset/covid19/caso.csv.gz')
src = pd.read_csv(url)

In [7]:
#hide
df = src[(src['place_type'] == 'state')]
df = df.rename(columns={
     "state": STATE_COLUMN, 
     "date":DATE_COLUMN, 
     "confirmed":CONFIRMED_CASES
})
# hold a backup
backup = df.copy()

In [8]:
#hide
url_cases = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
url_deaths = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
url_recoveries = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

src_cases = pd.read_csv(url_cases)
df_cases = src_cases[(src_cases['Country/Region'] == 'Brazil')]
df_cases['base'] = 'cases'

src_recoveries = pd.read_csv(url_recoveries)
df_recoveries = src_recoveries[(src_recoveries['Country/Region'] == 'Brazil')]
df_recoveries['base'] = 'recoveries'

src_deaths = pd.read_csv(url_deaths)
df_deaths = src_deaths[(src_deaths['Country/Region'] == 'Brazil')]
df_deaths['base'] = 'deaths'

df = [df_cases, df_deaths, df_recoveries]
df = pd.concat(df)

df = df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1)

# Add active cases row
df = df.set_index('base')
new_row = df.loc['cases'] - (df.loc['recoveries'] + df.loc['deaths'])
new_row.name = 'actives'
df = df.append([new_row])
# Reset index after added new row
df = df.reset_index()
df = df.rename(columns={"index":"base"})

# Melt structure (unpivot)
dt_cols = list(df.columns[~df.columns.isin(['base','index'])])
df = df.melt(id_vars=['base'], value_vars=dt_cols)
df = df.rename(columns={
     "base": STATE_COLUMN, 
     "variable":DATE_COLUMN, 
     "value":ACTIVE_CASES
})
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')
df = df.sort_values(by=[DATE_COLUMN])

df = df[df[STATE_COLUMN].isin(['actives'])]

/home/cleber/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/cleber/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/cleber/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [9]:
#hide
df[UPPER_BOUND_ICU_NEED_STR] = round(df[ACTIVE_CASES] * (UPPER_BOUND_ICU_NEED),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[ACTIVE_CASES] * (LOWER_BOUND_ICU_NEED),0)
df[ACTIVE_CASES] = round(df[ACTIVE_CASES] * (CASES_NEED_ICU),0)

In [10]:
#hide_input
selection = alt.selection_single(fields=[STATE_COLUMN], name=' ')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(ACTIVE_CASES+':Q', axis=alt.Axis(
        title=["Estimativa de casos que precisam de UTI: {:.0f}".format(CASES_NEED_ICU*100)+"% (variando entre {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"% e {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%)"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        ACTIVE_CASES
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Achate a curva - Brasil",
        "Casos ativos que estima-se que precisem de UTI (ver premissas)"
        
    ]
).add_selection(
    selection
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    opacity = alt.condition(selection, alt.value(0.2), alt.value(0.5))
)

x0line = alt.Chart(pd.DataFrame({'y': [TOTAL_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)

text0 = x0line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("Total de "+"{:.0f}".format(TOTAL_ICU)+" UTIs do país")   
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='darkorange', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=-10, color='darkorange', strokeWidth=1).encode(
    text=alt.value("Estimativa para COVID19: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU)+" UTIs")
)
text2 = x1line.mark_text(align='left', x=5, dy=10, color='darkorange', strokeWidth=1).encode(
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% do total de UTIs do país")   
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x0line + text0 + x1line + text1 + text2 + shades
plot

alt.LayerChart(...)

In [11]:
#hide_input
TOTAL_CASES = int(df_cases[pd.to_datetime(max(df[DATE_COLUMN])).strftime("%-m/%-d/%y")])
TOTAL_RECOVERIES = int(df_recoveries[pd.to_datetime(max(df[DATE_COLUMN])).strftime("%-m/%-d/%y")])
TOTAL_DEATHS = int(df_deaths[pd.to_datetime(max(df[DATE_COLUMN])).strftime("%-m/%-d/%y")])
ACTIVES_RATE = 1 - ((TOTAL_DEATHS + TOTAL_RECOVERIES) / TOTAL_CASES)
HTML(
    f'<small class="float-right">Última atualização em {pd.to_datetime(max(df[DATE_COLUMN])).strftime("%d/%m/%Y")}</small><br>'+
    f'<small class="float-right">Total de casos confirmados: {TOTAL_CASES}</small><br>'+
    f'<small class="float-right">Total de recuperados: {TOTAL_RECOVERIES}, {int(round(TOTAL_RECOVERIES / TOTAL_CASES, 2) * 100)}% dos casos</small><br>'+
    f'<small class="float-right">Total de mortes: {TOTAL_DEATHS}, {int(round(TOTAL_DEATHS / TOTAL_CASES, 2) * 100)}% dos casos</small><br>'
    f'<small class="float-right">Total de ativos: {TOTAL_CASES - (TOTAL_DEATHS + TOTAL_RECOVERIES)}, {int(round(ACTIVES_RATE, 2) * 100)}% dos casos</small>'
)

In [12]:
#hide
# get only states
df = backup[(backup[STATE_COLUMN] != 'Total')]
cols = df.columns[~df.columns.isin([DATE_COLUMN, STATE_COLUMN, CONFIRMED_CASES])]
df = df.drop(cols, axis=1)

In [13]:
#hide
# add lower and upper bounds
df[ACTIVE_CASES] = round(df[CONFIRMED_CASES] * (ACTIVES_RATE),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[ACTIVE_CASES] * (LOWER_BOUND_ICU_NEED),0)
df[CASES_NEED_ICU_COLUMN] = round(df[ACTIVE_CASES] * (CASES_NEED_ICU),0)
df[UPPER_BOUND_ICU_NEED_STR] = round(df[ACTIVE_CASES] * (UPPER_BOUND_ICU_NEED),0)
src_icus = src_icus.set_index(STATE_COLUMN)
df = df.set_index(STATE_COLUMN)
df[ICUS_COLUMN] = src_icus[ICUS_COLUMN].astype(float)
df[AVAILABLE_ICUS_COLUMN] = df[ICUS_COLUMN] * AVAILABLE_PERCENTAGE_ICU
src_icus = src_icus.reset_index()
df = df.reset_index()

In [14]:
#hide
# copy latest value on empty last items
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df = df.sort_values(by=[DATE_COLUMN])
LAST_DATE = max(df[DATE_COLUMN])
while ((df.groupby(STATE_COLUMN).last()[DATE_COLUMN] < LAST_DATE).any()):
    copied_registers = []
    for row in df.groupby(STATE_COLUMN).last().iterrows():
        if row[1][DATE_COLUMN] < LAST_DATE:
            copied_registers.append([row[0], row[1][DATE_COLUMN] + pd.DateOffset(days=1), 
                          row[1][CONFIRMED_CASES],
                          row[1][ACTIVE_CASES],
                          row[1][LOWER_BOUND_ICU_NEED_STR],
                          row[1][CASES_NEED_ICU_COLUMN],
                          row[1][UPPER_BOUND_ICU_NEED_STR],
                          row[1][ICUS_COLUMN],
                          row[1][AVAILABLE_ICUS_COLUMN]])
    for row in copied_registers:
        df.loc[row[0]] = [row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8]]
        df = df.set_index(STATE_COLUMN)
        df = df.reset_index()
df = df.sort_values(by=[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')
df = df.astype({ACTIVE_CASES: int, 
                LOWER_BOUND_ICU_NEED_STR: int, 
                CASES_NEED_ICU_COLUMN: int, 
                UPPER_BOUND_ICU_NEED_STR: int, 
                ICUS_COLUMN: int, 
                AVAILABLE_ICUS_COLUMN: int})

In [15]:
#hide_input
opt = df[STATE_COLUMN].unique()
opt.sort()
input_dropdown = alt.binding_select(options=opt)
selection = alt.selection_single(
    fields=[STATE_COLUMN], 
    init={STATE_COLUMN:'AC'}, 
    bind=input_dropdown, 
    name=" ")
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='Paired', reverse=True), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CASES_NEED_ICU_COLUMN+':Q', axis=alt.Axis(
        title=["Estimativa de casos que precisam de UTI: {:.0f}".format(CASES_NEED_ICU*100)+"% (variando entre {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"% e {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%)"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        CASES_NEED_ICU_COLUMN
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Achate a curva por estado",
        "Estimativa de casos ativos que precisem de UTI (ver premissas)"
    ]
).add_selection(
    selection
).transform_filter(
    selection
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    color=color,
    opacity = alt.condition(selection, alt.value(0.2), alt.value(0.5))
).transform_filter(
    selection
)

x1line = alt.Chart(df.groupby(DATE_COLUMN).last()).mark_rule(color='#e42726', strokeWidth=2).encode(
    y=ICUS_COLUMN+':Q'
).transform_filter(
    selection
)

text1 = alt.Chart(df.groupby(DATE_COLUMN).last()).mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    y=ICUS_COLUMN+':Q',
    text=alt.value("Total de leitos de UTI")   
).transform_filter(
    selection
)

x2line = alt.Chart(df.groupby(DATE_COLUMN).last()).mark_rule(color='darkorange', strokeWidth=2).encode(
    y=AVAILABLE_ICUS_COLUMN+':Q'
).transform_filter(
    selection
)

text2 = alt.Chart(df.groupby(DATE_COLUMN).last()).mark_text(align='left', x=5, dy=10, color='darkorange', strokeWidth=1).encode(
    y=AVAILABLE_ICUS_COLUMN+':Q',
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% dos leitos de UTI")   
).transform_filter(
    selection
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + x2line + text2 + shades
plot

alt.LayerChart(...)

In [16]:
#hide_input
df = df.groupby(STATE_COLUMN).last()
df = df.reset_index().drop(1, axis=0)
df = df.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] ) ])
df

,Estado,Data,Casos confirmados,Ativos,1% dos ativos,5% dos ativos,12% dos ativos,UTIs,35% das UTIs
0,AC,05/31/20,6072,3290,33,164,395,58,20
2,AM,05/31/20,41378,22421,224,1121,2691,450,157
3,AP,05/31/20,9602,5203,52,260,624,78,27
4,BA,05/31/20,18392,9966,100,498,1196,1573,550
5,CE,05/31/20,48489,26275,263,1314,3153,1002,350
6,DF,05/31/20,8722,4726,47,236,567,997,348
7,ES,05/31/20,13690,7418,74,371,890,804,281
8,GO,05/31/20,3726,2019,20,101,242,1200,420
9,MA,05/31/20,35297,19126,191,956,2295,682,238
10,MG,05/31/20,10464,5670,57,284,680,3151,1102


In [17]:
#hide_input
print("Premissas :")
print("- O percentual de casos ativos do país está senso utilizado para estimar o percentual em cada estado.")
print("- O Brasil tem "+"{:.0f}".format(TOTAL_ICU)+" leitos de UTI adulto SUS e não SUS (fonte: painel de insumos e leitos)")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)")
print("- Estima-se que cerca de "+"{:.0f}".format(CASES_NEED_ICU*100)+"% dos infectados por covid-19 venham a precisar de tratamento intensivo. No pior caso registrado, na Itália, cerca de 12% precisaram de UTI e no melhor caso, China, cerca de 1% dos infectados precisaram de UTI (fonte: artigo the lancet)")
print("- As estimativas são feitas para o número de infectados. Nesta simplificação, estas estimativas estão sendo extrapolada para o número de ativos.")

Premissas :
- O percentual de casos ativos do país está senso utilizado para estimar o percentual em cada estado.
- O Brasil tem 34318 leitos de UTI adulto SUS e não SUS (fonte: painel de insumos e leitos)
- 35% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)
- Estima-se que cerca de 5% dos infectados por covid-19 venham a precisar de tratamento intensivo. No pior caso registrado, na Itália, cerca de 12% precisaram de UTI e no melhor caso, China, cerca de 1% dos infectados precisaram de UTI (fonte: artigo the lancet)
- As estimativas são feitas para o número de infectados. Nesta simplificação, estas estimativas estão sendo extrapolada para o número de ativos.


In [18]:
#hide_input
HTML(f'<small class="float-right">Última atualização em {pd.to_datetime(LAST_DATE).strftime("%d/%m/%Y")}</small>')

Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html) and [Joao B. Duarte](https://github.com/github/covid19-dashboard/blob/master/_notebooks/2020-03-19-estimating_infected.ipynb), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). 

Dados:
- covid19: [CSSEGISandData](https://github.com/CSSEGISandData/COVID-19)
- quantidade de leitos: [painel de insumos e leitos](https://covid-insumos.saude.gov.br/paineis/insumos/painel_leitos.php), [elpais](https://brasil.elpais.com/brasil/2020-04-28/sem-transparencia-sobre-fila-para-utis-justica-opera-para-garantir-atendimento-a-pacientes-de-covid-19.html), [elpais2](https://brasil.elpais.com/sociedade/2020-04-15/sem-leitos-de-uti-municipios-pequenos-temem-por-estrutura-limitada-para-transferir-pacientes-graves-com-a-covid-19.html), [bcc](https://www.bbc.com/portuguese/brasil-52137553) e [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- leitos disponíveis: [artigo oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- pacientes que precisam de UTI: [artigo the lancet](https://linkinghub.elsevier.com/retrieve/pii/S2213260020301612)

In [19]:
#hide
save(plot,"../images/brazil-flatten-the-curve.png")

ValueError: No enabled saver found that supports format='png'